In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# 1. Завантаження даних
try:
    df = pd.read_csv("boston.csv")
    print("Дані успішно завантажено\n")
except FileNotFoundError:
    print("Файл boston.csv не знайдено. Перевірте шлях до файлу.")
    exit()

# 2. Попередня підготовка даних
if 'CHAS' in df.columns:
    le = LabelEncoder()
    df['CHAS'] = le.fit_transform(df['CHAS'])

scaler = StandardScaler()
df[df.columns] = scaler.fit_transform(df)
print("Дані підготовлені: категоріальні змінні закодовані, числові масштабовані\n")

# Визначаємо вхідні ознаки та цільову змінну для класифікації
if 'MEDV' not in df.columns:
    print("Колонка MEDV відсутня у файлі.")
    exit()

X = df.drop(columns=['MEDV'])
y_binary = (df['MEDV'] > df['MEDV'].median()).astype(int)  # Бінарна класифікація
y_multi = pd.qcut(df['MEDV'], q=3, labels=[0, 1, 2])  # Множинна класифікація

# 3. Розбиття на навчальну та тестову вибірки
X_train, X_test, y_train_binary, y_test_binary = train_test_split(X, y_binary, test_size=0.2, random_state=42)
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(X, y_multi, test_size=0.2, random_state=42)
print("Дані розбиті: 80% - навчальна вибірка, 20% - тестова\n")

# 4. Побудова моделей логістичної регресії
# Бінарна класифікація
log_reg_binary = LogisticRegression(max_iter=1000, solver='lbfgs')
log_reg_binary.fit(X_train, y_train_binary)
y_pred_binary = log_reg_binary.predict(X_test)

# Множинна класифікація
log_reg_multi = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
log_reg_multi.fit(X_train_multi, y_train_multi)
y_pred_multi = log_reg_multi.predict(X_test_multi)

# 5. Оцінка якості моделей
print("Точність бінарної моделі:", accuracy_score(y_test_binary, y_pred_binary))
print("Confusion Matrix (бінарна класифікація):")
print(confusion_matrix(y_test_binary, y_pred_binary))
print("Classification Report (бінарна класифікація):")
print(classification_report(y_test_binary, y_pred_binary))

print("Точність множинної моделі:", accuracy_score(y_test_multi, y_pred_multi))
print("Confusion Matrix (множинна класифікація):")
print(confusion_matrix(y_test_multi, y_pred_multi))
print("Classification Report (множинна класифікація):")
print(classification_report(y_test_multi, y_pred_multi))


# Оптимізація множинної моделі
param_grid_multi = {
    'penalty': ['l2'],  # 'l1' не підтримується solver='lbfgs'
    'C': np.logspace(-3, 3, 10),
    'solver': ['lbfgs', 'saga'],
    'multi_class': ['multinomial']
}

grid_search_multi = GridSearchCV(LogisticRegression(max_iter=1000), param_grid_multi, cv=5, scoring='accuracy')
grid_search_multi.fit(X_train_multi, y_train_multi)
print("Найкращі параметри для множинної класифікації:", grid_search_multi.best_params_)

# Перенавчання з кращими параметрами
optimized_log_reg_multi = LogisticRegression(**grid_search_multi.best_params_, max_iter=1000)
optimized_log_reg_multi.fit(X_train_multi, y_train_multi)
opt_pred_multi = optimized_log_reg_multi.predict(X_test_multi)
opt_accuracy_multi = accuracy_score(y_test_multi, opt_pred_multi)
print("Оптимізована точність множинної моделі:", opt_accuracy_multi)


# 6. Підбір оптимальних параметрів
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': np.logspace(-3, 3, 10),
    'solver': ['liblinear', 'saga']
}

grid_search_binary = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5, scoring='accuracy')
grid_search_binary.fit(X_train, y_train_binary)
print("Найкращі параметри для бінарної класифікації:", grid_search_binary.best_params_)

# 7. Перенавчання моделей з оптимальними параметрами
optimized_log_reg_binary = LogisticRegression(**grid_search_binary.best_params_, max_iter=1000)
optimized_log_reg_binary.fit(X_train, y_train_binary)
opt_pred_binary = optimized_log_reg_binary.predict(X_test)
opt_accuracy_binary = accuracy_score(y_test_binary, opt_pred_binary)
print("Оптимізована точність бінарної моделі:", opt_accuracy_binary)

# 8. Вибір найкращої моделі та аналіз
models = {
    "Бінарна початкова": accuracy_score(y_test_binary, y_pred_binary),
    "Бінарна оптимізована": opt_accuracy_binary
}

best_model = max(models, key=models.get)
print("Найкраща модель:", best_model)

Дані успішно завантажено

Дані підготовлені: категоріальні змінні закодовані, числові масштабовані

Дані розбиті: 80% - навчальна вибірка, 20% - тестова

Точність бінарної моделі: 0.8725490196078431
Confusion Matrix (бінарна класифікація):
[[52  8]
 [ 5 37]]
Classification Report (бінарна класифікація):
              precision    recall  f1-score   support

           0       0.91      0.87      0.89        60
           1       0.82      0.88      0.85        42

    accuracy                           0.87       102
   macro avg       0.87      0.87      0.87       102
weighted avg       0.88      0.87      0.87       102

Точність множинної моделі: 0.7745098039215687
Confusion Matrix (множинна класифікація):
[[31  5  1]
 [ 7 26  3]
 [ 0  7 22]]
Classification Report (множинна класифікація):
              precision    recall  f1-score   support

           0       0.82      0.84      0.83        37
           1       0.68      0.72      0.70        36
           2       0.85      0.76